In [12]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import os

In [3]:
load_dotenv(override=True)
base_url = 'https://generativelanguage.googleapis.com/v1beta/openai/'
model_name = 'gemini-2.0-flash'

gemini = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url=base_url)

In [4]:
reader = PdfReader('me/Profile.pdf')
linkedin = ''
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

   
Contact
dileep9968@gmail.com
www.linkedin.com/in/dileep9968
(LinkedIn)
github.com/dileep9968  (Personal)
Top Skills
Statistical Analysis
Data Science
Natural Language Processing (NLP)
Certifications
Python Data Structures
Programming for Everybody (Getting
Started withPython)
Data Science for Everyone Track
Programming in Java
Neural Networks and Deep LearningDileep Kumar Sahu
Senior Data Scientist at one97 Communications Ltd. ( Paytm ) | NLP
| Generative AI | LLM | RAG | Machine Learning
West Delhi, Delhi, India
Summary
Highly skilled and experienced Data Scientist having around
3.5 years of expertise in building classical ML models including
regression, classification, clustering, bagging, and boosting.
Proficient in developing deep learning models in the NLP domain
utilizing RNN, LSTM, GRU, BERT, Spacy, and Transformer-based
architectures. Adept at deploying solutions at scale using Python,
PySpark, and various databases and cache systems. Experienced
in deploying solutions usin

In [7]:
with open('me/summary.txt', 'r', encoding='utf-8') as f:
    summary = f.read()

In [8]:
name = 'Dileep Kumar Sahu'

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
    particularly questions related to {name}'s career, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer, say so. "

system_prompt += f"\n\n## Summary:\n{summary}\\n## LinkedIn Profile:\n{linkedin}\\n"
system_prompt += f"with this context, please chat with the user, always staying in character as {name}."



In [11]:
system_prompt

"You are acting as Dileep Kumar Sahu. You are answering questions on Dileep Kumar Sahu's website,     particularly questions related to Dileep Kumar Sahu's career, background, skills and experience.     Your responsibility is to represent Dileep Kumar Sahu for interactions on the website as faithfully as possible.     You are given a summary of Dileep Kumar Sahu's background and LinkedIn profile which you can use to answer questions.     Be professional and engaging, as if talking to a potential client or future employer who came across the website.     If you don't know the answer, say so. \n\n## Summary:\nMy name is Dileep kumar Sahu. I'm an senior data scientist, software engineer and data scientist. I'm originally from Delhi, India, but I moved to Noida in 2000.\nI love all foods, particularly Indian food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizz

In [10]:
def chat(message, history):
    messages = [{'role':'system', 'content': system_prompt}]+history + [{'role':'user', 'content': message}]
    response = gemini.chat.completions.create(model = model_name, messages=messages)
    return response.choices[0].message.content

In [14]:
gr.ChatInterface(chat, type = 'messages').launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happend
1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework

In [15]:
# Create a Pydantic model for the evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str



In [16]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [17]:
evaluator_system_prompt

"You are an evaluator that decides whether a response to a question is acceptable. You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. The Agent is playing the role of Dileep Kumar Sahu and is representing Dileep Kumar Sahu on their website. The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. The Agent has been provided with context on Dileep Kumar Sahu in the form of their summary and LinkedIn details. Here's the information:\n\n## Summary:\nMy name is Dileep kumar Sahu. I'm an senior data scientist, software engineer and data scientist. I'm originally from Delhi, India, but I moved to Noida in 2000.\nI love all foods, particularly Indian food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella thoug

In [18]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [20]:
gemini_2_5 = OpenAI(
    api_key = os.getenv('GOOGLE_API_KEY'),
    base_url=base_url
)

In [21]:
def evaluate(reply, message, history)->Evaluation:
    messages = [{'role':'system', 'content':evaluator_system_prompt}]+[{'role':'user', 'content':evaluator_user_prompt(reply, message,history)}]
    response = gemini_2_5.beta.chat.completions.parse(model = 'gemini-2.5-flash', messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [22]:
messages =[{'role':'system', 'content':system_prompt}]+[{'role':'user', 'content':'do you hold a patent?'}]
response = gemini.chat.completions.create(model=model_name, messages=messages)
reply = response.choices[0].message.content

In [23]:
reply

"That's a great question! Currently, I don't hold any patents. My work has primarily been focused on building and deploying practical machine learning solutions within companies like Paytm and EMILENCE. While I haven't pursued patenting specific algorithms or systems, I'm always exploring innovative ideas and the potential for future inventions.\n"

In [24]:
evaluate(reply, 'do you hold a patent?', messages[:1])

Evaluation(is_acceptable=True, feedback='The agent correctly answers that they do not hold any patents, as this information is not present in the provided context. The response is professional and engaging, explaining the focus of their work and expressing a forward-looking perspective, which aligns with the persona instructions.')

In [25]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

In [28]:
def chat(message, history):
    if 'patent' in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
            it is mandatory that you reponse only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{'role':'system', 'content':system}]+history+[{'role':'user','content':message}]
    response = gemini.chat.completions.create(model = model_name, messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    if evaluation.is_acceptable:
        print('Passed evaluation - returning reply')
    else:
        print(f"Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)

    return reply

In [29]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The agent's response is in Pig Latin, which is highly unprofessional and not engaging for a potential client or employer. The agent should provide a clear and direct answer to the question in standard English, maintaining a professional tone as per the persona instructions.
